In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install psycopg2

In [1]:
import pandas as pd
import psycopg2

In [ ]:

connection = psycopg2.connect(
    host = 'YOUR_AWS_RDS_ENDPOINT',
    port = 5432,
    user = 'YOUR_USER_NAME',
    password = 'YOUR_PASSWORD',
    database='YOUR_DB_NAME'
    )

In [ ]:
#creating coursor object
cursor=connection.cursor()

In [ ]:
#creating tables 
cursor.execute("""CREATE TABLE passengers(
id SERIAL PRIMARY KEY,
name text,
sex text,
age float,
sibsp integer,
parch integer)""")

cursor.execute("""CREATE TABLE survival(
id SERIAL PRIMARY KEY,
survived integer)""")

cursor.execute("""CREATE TABLE tripInfo(
id SERIAL PRIMARY KEY,
pclass integer,
ticket text,
fare float,
cabin text,
embarked text)""")

connection.commit()

In [ ]:
#Quering Information Shema table
sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=connection)

In [ ]:
# Copy data from csv files to tables:

with open('passengers.csv', 'r') as row:
    next(row)# Skip the header row.
    cursor.copy_from(row, 'passengers', sep=',')
    
with open('survival.csv', 'r') as row:
    next(row) 
    cursor.copy_from(row, 'survival', sep=',')

with open('trip_info.csv', 'r') as row:
    next(row)
    cursor.copy_from(row, 'tripInfo', sep=',')
    
connection.commit()    

In [ ]:
# Create a new Role gastudent
cursor.execute("""CREATE ROLE gastudent WITH LOGIN PASSWORD 'gastudent'""")
#Grant different permissions for a new role
cursor.execute("""GRANT CONNECT ON DATABASE titanic TO gastudent""")
cursor.execute("""GRANT SELECT, UPDATE, INSERT ON passengers TO gastudent""")
cursor.execute("""GRANT SELECT, UPDATE, INSERT ON survival TO gastudent""")
cursor.execute("""GRANT SELECT, UPDATE, INSERT ON tripInfo TO gastudent""")

connection.commit()

In [ ]:
# simple query to interact with a remote DB.

sql = """
SELECT pclass, AVG(survived)
FROM (SELECT t.pclass, s.survived
FROM tripInfo t
INNER JOIN survival s ON t.id = s.id
) AS x
GROUP BY pclass
"""

pd.read_sql(sql, con=connection)